In [5]:
pip install -U datasets pandas openpyxl


^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
from datasets import load_dataset
import pandas as pd

# Load the dataset with trust_remote_code
dataset = load_dataset("casehold/casehold", trust_remote_code=True)

# Convert the training data to a DataFrame
df = pd.DataFrame(dataset['train'])

# Save to Excel file
df.to_excel("casehold_train.xlsx", index=False)


train.csv:   0%|          | 0.00/85.3M [00:00<?, ?B/s]

     --------------------------------------- 11.6/11.6 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.0
    Uninstalling pandas-2.2.0:
      Successfully uninstalled pandas-2.2.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\manan\\anaconda3\\Lib\\site-packages\\~andas.libs\\msvcp140-fa0758dedafbbe194d3ee96e3dc2b9a3.dll'
Consider using the `--user` option or check the permissions.



val.csv:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

C:\Users\manan\AppData\Local\Temp\ipykernel_12484\1369951202.py:11: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel("casehold_train.xlsx", index=False)


In [7]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [8]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\manan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load your Excel file
df = pd.read_excel("casehold_train.xlsx")

# Preprocessing functions
def tokenize_text(text):
    return word_tokenize(str(text))

def lowercase_tokens(tokens):
    return [token.lower() for token in tokens]

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]

def remove_punctuation(tokens):
    return [token for token in tokens if token.isalnum()]

def preprocess_text(text):
    tokens = tokenize_text(text)
    tokens_lower = lowercase_tokens(tokens)
    tokens_no_stop = remove_stopwords(tokens_lower)
    tokens_no_punct = remove_punctuation(tokens_no_stop)
    return {
        'original': text,
        'tokenized': tokens,
        'lowercase': tokens_lower,
        'no_stopwords': tokens_no_stop,
        'no_punctuation': tokens_no_punct
    }

# Apply to 'citing_prompt' column
processed = df['citing_prompt'].apply(preprocess_text)

# Normalize to new DataFrame
processed_df = pd.json_normalize(processed)

# Optional: Add example_id back for reference
processed_df['example_id'] = df['example_id']

# Save the result
processed_df.to_excel("casehold_citing_prompt_preprocessed.xlsx", index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\manan\AppData\Local\Temp\ipykernel_12484\4131426476.py:50: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  processed_df.to_excel("casehold_citing_prompt_preprocessed.xlsx", index=False)


In [11]:
import pandas as pd

# Load preprocessed citing_prompt data
preprocessed_samples = pd.read_excel("casehold_citing_prompt_preprocessed.xlsx")

# Convert string representations of lists back into actual lists (Excel saves them as strings)
import ast

for col in ['tokenized', 'lowercase', 'no_stopwords', 'no_punctuation']:
    preprocessed_samples[col] = preprocessed_samples[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Focus on Sample 1 for detailed analysis
sample_idx = 1
sample = preprocessed_samples.iloc[sample_idx]

# Original text
print(f"ORIGINAL TEXT (truncated):\n{sample['original'][:500]}...\n")

# Step 1: Tokenization
print("STEP 1: TOKENIZATION (first 78 tokens)")
print(sample['tokenized'][:78])
print(f"Total tokens: {len(sample['tokenized'])}\n")

# Step 2: Lowercase
print("STEP 2: LOWERCASE (first 78 tokens)")
print(sample['lowercase'][:78])
print(f"Total tokens: {len(sample['lowercase'])}\n")

# Step 3: Stopword Removal
print("STEP 3: STOPWORD REMOVAL (first 78 tokens)")
print(sample['no_stopwords'][:78])
print(f"Total tokens: {len(sample['no_stopwords'])}\n")

# Step 4: Punctuation Removal
print("STEP 4: PUNCTUATION REMOVAL (first 78 tokens)")
print(sample['no_punctuation'][:78])
print(f"Total tokens: {len(sample['no_punctuation'])}\n")


ORIGINAL TEXT (truncated):
Colameta used customer information that he took from Protégé. Additionally, Colameta admits to having taken at least two Protégé proposals with him to Monument. This type of information may constitute trade secrets. See G.L.c. 266, §30 (defining “trade secret” as used in G.L.c. 93, §42, as including “anything tangible or intangible or electronically kept or stored, which constitutes, represents, evidences or records a secret scientific, technical, merchandising, production or management informat...

STEP 1: TOKENIZATION (first 78 tokens)
['Colameta', 'used', 'customer', 'information', 'that', 'he', 'took', 'from', 'Protégé', '.', 'Additionally', ',', 'Colameta', 'admits', 'to', 'having', 'taken', 'at', 'least', 'two', 'Protégé', 'proposals', 'with', 'him', 'to', 'Monument', '.', 'This', 'type', 'of', 'information', 'may', 'constitute', 'trade', 'secrets', '.', 'See', 'G.L.c', '.', '266', ',', '§30', '(', 'defining', '“', 'trade', 'secret', '”', 'as', 'used', 

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Excel file
df = pd.read_excel("casehold_train.xlsx")

# Strip whitespace and lower all column names
df.columns = df.columns.str.strip().str.lower()

# Print all column names to debug
print("✅ Available columns:\n", df.columns.tolist())

# Define best matching column function
def get_best_column_match(possible_names):
    for name in possible_names:
        for col in df.columns:
            if name.lower() in col.lower():
                return col
    return None

# Try to find correct column names
example_id_col = get_best_column_match(["example_id"])
citing_prompt_col = get_best_column_match(["citing_prompt"])
holding_cols = [get_best_column_match([f"holding_{i}"]) for i in range(5)]

# Check if any column is missing
if not all([example_id_col, citing_prompt_col] + holding_cols):
    print("❌ Error: One or more required columns not found.")
    print("🔍 Detected:", example_id_col, citing_prompt_col, holding_cols)
else:
    print("✅ Matched Columns:")
    print(f"Example ID Column: {example_id_col}")
    print(f"Citing Prompt Column: {citing_prompt_col}")
    print(f"Holding Columns: {holding_cols}")

    # TF-IDF Logic
    def find_best_holding(example_id):
        row = df[df[example_id_col] == example_id]

        if row.empty:
            return f"❌ No data found for example_id: {example_id}"

        citing_prompt = row[citing_prompt_col].values[0]
        holdings = [row[col].values[0] for col in holding_cols]

        corpus = [citing_prompt] + holdings

        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
        similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

        best_index = similarities.argmax()
        best_score = similarities[best_index]
        best_holding_col = holding_cols[best_index]

        return {
            'example_id': example_id,
            'best_holding': best_holding_col,
            'similarity_score': round(float(best_score), 4),
            'holding_text': holdings[best_index]
        }

    # Try it!
    example_id_input = 3
    result = find_best_holding(example_id_input)
    print("\n📌 Best Holding Result:")
    print(result)


✅ Available columns:
 ['example_id', 'citing_prompt', 'holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4']
✅ Matched Columns:
Example ID Column: example_id
Citing Prompt Column: citing_prompt
Holding Columns: ['holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4']

📌 Best Holding Result:
{'example_id': 3, 'best_holding': 'holding_0', 'similarity_score': 0.1123, 'holding_text': 'holding that plaintiff stated a  349 claim where plaintiff alleged facts plausibly suggesting that defendant intentionally registered its corporate name to be confusingly similar to plaintiffs commscope trademark'}
